# Deploying GPT-2 to ODSP

This notebook will showcase an example of how to deploy a language transformer model from Huggingface on the Open Data Science Platform.

In this notebook, we will show how to access GPT-2 via Hugging Face, create a pipeline for the model, and register the model using `MLFlow`, and use that model as it is automatically deployed to the platform.

In [ ]:
# Import all required packages
from transformers import pipeline
import requests
import mlflow

## Access GPT-2 and Create a Transformers Pipeline

In [ ]:
# Create a text generation transformers pipeline object for GPT-2
model_id = 'gpt2'
model = pipeline('text-generation', model_id)

In [ ]:
# Create a model signature & config 
data = ['How are you today?']
signature_output = mlflow.transformers.generate_signature_output(model, data)
signature = mlflow.models.infer_signature(data, signature_output)
model_config = {
    'max_new_tokens' : 50,
    'do_sample' : False,
    'return_full_text' : False
}

## Log and Register GPT-2 Model Pipeline with MLflow

In [ ]:
# Create MLflow experiment
mlflow.set_experiment('GPT2Tutorial')

# Create your register your model with MLflow
with mlflow.start_run() as run:
    model_info = mlflow.transformers.log_model(
        model,
        'gpt2',
        model_config = model_config,
        signature = signature,
        registered_model_name = 'gpt2'
    )

## Make Predictions with Your Deployed Model

In [ ]:
import requests

In [ ]:
# User and default API key, if the default environment file is used (NOT RECOMMENDED FOR ANYTHING OTHER THAN TESTING PURPOSES)
user = 'odsp'
key = 'odsp-odsp'

# Make a prediction using the deployed model

# Note that the model is being deployed as an MLflow pyfunc object with a version number
# Make sure that you're careful to point to the correct model version (in the example, it's 1) when making your request
with requests.Session() as sess:
    resp = sess.post('http://model-server:4488/models/predict/gpt2/pyfunc/1', json = {'data' : ['Tell me about yourself.']}, auth = ('odsp', 'odsp-odsp'))

In [ ]:
# Return the model's predictions
print(resp.json()['prediction'][0])